**Modelo tipo 1 e 2**

* Tipo 1: demanda <= oferta

* Tipo 2: demanda >= oferta

* fim no cliente

In [19]:
import gurobipy as gp
import numpy as np
import pandas as pd
from math import *

In [20]:
df_ori_dest = pd.read_csv('../dados/origem_porto.csv')
df_ori_trans = pd.read_csv('../dados/origem_transbordo.csv')
df_trans_porto = pd.read_csv('../dados/transbordo_porto.csv')

In [21]:
df_supply = pd.read_csv('../dados/supply.csv')
df_cap_port = pd.read_csv('../dados/cap_porto.csv')
df_cap_trans = pd.read_csv('../dados/cap_transbordo.csv')
df_demand = pd.read_csv('../dados/demand.csv')

In [22]:
qnt_orig = df_supply.shape[0]
qnt_trans = df_cap_trans.shape[0]
qnt_port = df_cap_port.shape[0]
qnt_cli = df_demand.shape[0]

In [23]:
df_supply

,0
0,45061
1,3999
2,28670
3,42047
4,10549
...,...
95,2389
96,17974
97,16647
98,5862


In [24]:
df_demand

,0
0,663
1,535
2,613
3,900
4,987
...,...
95,632
96,313
97,911
98,844


In [25]:
df_ori_dest

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,70.080183,81.643596,107.650516,59.012243,73.977813,58.930630,127.545694,113.803648,103.001359,112.556901,...,58.333551,99.390398,116.477363,122.508292,67.209523,100.139295,59.881910,48.995020,107.489758,78.790862
1,73.559815,86.763384,113.078352,64.993292,79.653972,61.184678,132.889127,117.023020,101.734965,119.281583,...,60.325305,103.596139,123.216622,128.966451,73.045914,106.180465,59.075328,55.531296,109.018369,85.636927
2,73.034348,84.872573,111.900527,63.589081,77.665951,59.906273,131.746180,116.603890,102.635978,117.178128,...,59.111068,102.847522,121.112814,126.880101,71.701604,104.141811,58.623135,53.435449,109.121525,83.847481
3,75.822108,87.117759,115.456603,67.343466,80.287643,60.480847,135.267675,119.238759,103.003223,120.550630,...,59.537417,105.922659,124.496965,130.029117,75.408222,106.990243,57.477425,56.702564,110.956626,87.796856
4,76.833326,83.262284,115.189555,66.599988,76.507610,56.859687,135.070518,120.436539,106.131270,117.169388,...,55.971879,106.523477,121.125495,126.473498,74.793968,103.282819,54.823061,53.266545,113.004977,86.021244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,76.781664,90.430579,117.438256,69.728066,83.737495,62.926192,137.182623,119.963941,101.528794,124.128037,...,61.886916,107.204537,128.074079,133.593772,77.689021,110.498854,58.605665,60.279244,110.820879,90.798414
96,72.680341,83.251984,110.962163,62.464172,75.965129,58.817576,130.835102,116.324402,103.472508,115.389289,...,58.078457,102.287063,119.323962,125.101277,70.626907,102.399250,58.247912,51.650518,109.289487,82.365041
97,71.240079,80.737243,108.518459,59.763802,73.191387,57.691039,128.427486,114.976672,104.259369,112.148473,...,57.080547,100.462164,116.077216,121.975211,67.995482,99.458373,58.609160,48.484224,108.760581,79.174137
98,71.446763,81.248025,108.933800,60.231447,73.746116,57.947839,128.835871,115.168885,104.044052,112.776735,...,57.313286,100.749194,116.706264,122.587360,68.448778,100.041463,58.586442,49.101234,108.792588,79.755306


In [26]:
N = [i for i in range(qnt_orig)]
M = [i + qnt_orig + qnt_trans for i in range(qnt_port)]
K = [i + qnt_orig for i in range(qnt_trans)]
O = [i + qnt_orig + qnt_trans + qnt_port for i in range(qnt_cli)]

supply = {}
for i in N:
    supply[i] = df_supply['0'][i]

demand = {}
for i in O:
    demand[i] = df_demand['0'][i - O[0]]

cap_transbordo = {}
for i in K:
    cap_transbordo[i] = df_cap_trans['0'][i - K[0]]

cap_port = {}
for i in M:
    cap_port[i] = df_cap_port['0'][i - M[0]]

cost = {}
for i in N:
    for j in M:
        cost[i, j] = df_ori_dest[str(j - M[0])][i]
    for k in K:
        cost[i, k] = df_ori_trans[str(k - K[0])][i]

for j in M:
    for k in K:
        cost[k, j] = df_trans_porto[str(j - M[0])][k - K[0]]
    for o in O:
        cost[j, o] = 0


In [27]:
oferta_total = 0
for i in supply:
    oferta_total += supply[i]
demanda_total = 0
for i in demand:
    demanda_total += demand[i]

print(oferta_total, demanda_total)

2678492 52641


In [28]:
m = gp.Model("probrema")

In [29]:
X = {}
for i in N:
    for j in M:
        X[i, j] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}".format(i, j))

for i in N:
    for k in K:
        X[i, k] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}".format(i, k))
                
for j in M:
    for k in K:
        X[k, j] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}".format(k, j))

for j in M:
    for o in O:
        X[j, o] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}".format(j, o))

In [30]:
m.setObjective(
    gp.quicksum((X[i, j] * cost[i, j]) for i in N for j in M) + \
    gp.quicksum((X[i, k] * cost[i, k]) for i in N for k in K) + \
    gp.quicksum((X[k, j] * cost[k, j]) for j in M for k in K),
    sense=gp.GRB.MINIMIZE
)

In [31]:
if oferta_total <= demanda_total:
    for i in N:
        m.addConstr(
            (gp.quicksum(X[i, j] for j in M) + gp.quicksum(X[i, k] for k in K)) == supply[i]
        )
else:
    for i in N:
        m.addConstr(
            (gp.quicksum(X[i, j] for j in M) + gp.quicksum(X[i, k] for k in K)) <= supply[i]
        )

In [32]:
if oferta_total <= demanda_total:
    for o in O:
        m.addConstr(
            gp.quicksum(X[j, o] for j in M) <= demand[o]
        )
else:
    for o in O:
        m.addConstr(
            gp.quicksum(X[j, o] for j in M) == demand[o]
        )

In [33]:
for j in M:
    m.addConstr(
        (gp.quicksum(X[i, j] for i in N) + gp.quicksum(X[k, j] for k in K)) <= cap_port[j]
    )

for k in K:
    m.addConstr(
        gp.quicksum(X[i, k] for i in N) <= cap_transbordo[k]
    )

for k in K:
    m.addConstr(
        gp.quicksum(X[i, k] for i in N) == gp.quicksum(X[k, j] for j in M)
    )

for j in M:
    m.addConstr(
        gp.quicksum(X[j, o] for o in O) == (gp.quicksum(X[i, j] for i in N) + gp.quicksum(X[k, j] for k in K))
    )

In [34]:
rest = m.addConstrs(
    gp.quicksum(X[i, j] for i in N) >= 0 for j in M
)

rest = m.addConstrs(
    gp.quicksum(X[i, k] for i in N) >= 0 for k in K
)

rest = m.addConstrs(
    gp.quicksum(X[k, j] for k in K) >= 0 for j in M
)

In [35]:
# Executa o modelo
m.update()
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 900 rows, 40000 columns and 140000 nonzeros
Model fingerprint: 0xb1e5e5d9
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 5e+04]
Presolve removed 300 rows and 0 columns
Presolve time: 0.08s
Presolved: 600 rows, 40000 columns, 110000 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.01s

Barrier statistics:
 AA' NZ     : 7.020e+04
 Factor NZ  : 1.016e+05 (roughly 17 MB of memory)
 Factor Ops : 2.372e+07 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.7603

In [36]:
m.write("out.sol")